Let's play with Bri's data. First, we'll write some code to wrangle the behaviour:

In [ ]:
from pathlib import Path
import pandas as pd
from pynapple import Ts, TsGroup, Tsd

def bri_spikes_to_pynapple(path_to_spikes):

    sampling_frequency = 30_000

    spikes_df = pd.read_pickle(path_to_spikes)

    spikes_dict = dict(zip(spikes_df['cluster_id'].values, spikes_df['firing_times'].values))
    spikes_dict_s = {key: Ts(t=value/sampling_frequency) for key, value in spikes_dict.items()}
    spikes_frame = TsGroup(data=spikes_dict_s)

    return spikes_frame

def bri_behaviour_to_pynapple(path_to_position):

    position_df = pd.read_pickle(path_to_position)

    Px = Tsd(t=position_df['synced_time'].values, d=position_df['position_x'].values)
    Py = Tsd(t=position_df['synced_time'].values, d=position_df['position_y'].values)

    return Px, Py


Then let's make some Sessions, and combine them into a SessionGroup:

In [ ]:
from object_scores.core import SingleSession, SessionGroup

active_projects_path = Path('/run/user/1000/gvfs/smb-share:server=cmvm.datastore.ed.ac.uk,share=cmvm/sbms/groups/CDBS_SIDB_storage/NolanLab/ActiveProjects/')

# of

path_to_spikes = active_projects_path / 'Bri/optetrode_recordings/chR2/1544/1544_2023-03-01_12-51-10_of/MountainSort/DataFrames/spatial_firing.pkl'
path_to_position = active_projects_path / 'Bri/optetrode_recordings/chR2/1544/1544_2023-03-01_12-51-10_of/MountainSort/DataFrames/position.pkl'

spikes = bri_spikes_to_pynapple(path_to_spikes)
Px, Py = bri_behaviour_to_pynapple(path_to_position)
of_session = SingleSession(spikes, Px, Py)

# obj 

path_to_spikes = active_projects_path / 'Bri/optetrode_recordings/chR2/1544/1544_2023-03-01_13-15-57_obj/MountainSort/DataFrames/spatial_firing.pkl'
path_to_position = active_projects_path /  'Bri/optetrode_recordings/chR2/1544/1544_2023-03-01_13-15-57_obj/MountainSort/DataFrames/position.pkl'
object_dict = {'A': [59.04081555, 43.91803363]}

spikes = bri_spikes_to_pynapple(path_to_spikes)
Px, Py = bri_behaviour_to_pynapple(path_to_position)
obj_session = SingleSession(spikes, Px, Py, object_dict)

# obj1

path_to_spikes = active_projects_path / 'Bri/optetrode_recordings/chR2/1544/1544_2023-03-01_13-36-53_obj1/MountainSort/DataFrames/spatial_firing.pkl'
path_to_position = active_projects_path /  'Bri/optetrode_recordings/chR2/1544/1544_2023-03-01_13-36-53_obj1/MountainSort/DataFrames/position.pkl'
object_dict = {'A': [34.85711347, 30.09364155]}

spikes = bri_spikes_to_pynapple(path_to_spikes)
Px, Py = bri_behaviour_to_pynapple(path_to_position)
obj1_session = SingleSession(spikes, Px, Py, object_dict)

session_group = SessionGroup({'of': of_session, 'obj': obj_session, 'obj1': obj1_session})

Then, like in the other example, we can compute some scores

In [ ]:
session_group.compute_information_content()
session_group.compute_ori()
session_group.compute_increase()

Then view some scores:

In [ ]:
session_group.scores

And plot some stuff:

In [ ]:
fig = session_group.plot_ori(cluster_id=17)